## Adam's Part

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#read in the data from the CSV file
#did not use ID as index because it will be easier to model (train/test data) this way 
df = pd.read_csv('train.csv', index_col = None)

#given that we would like to predict the review of each listing, we remove listings that have not been rated
df = df.dropna(subset=['review_scores_rating'])


In [ ]:
import math

#from the standpoint of agility, let us create a rough model, relating price of a suite to the number of bedrooms and bathrooms
#also tried it with some consideratin for the average review
#while no clear model arises from this attempt, it does appear that this model creates a lower bound
plt.scatter(df.bathrooms+df.bedrooms+0.2*df.review_scores_rating, df.log_price)

In [ ]:
#the next step is to convert many of our categorical data columns into numbers
#start with bed_type, where we can use: Real Bed = 4, Pull-out Sofa or Futon = 3, Couch = 2, Airbed = 1, in order of luxury
bed_data = [['Real Bed',4],['Pull-out Sofa',3],['Futon',3],['Couch',2],['Airbed',1]]
df_bed_data = pd.DataFrame(bed_data,columns=['bed_type','bed_val'])
df = df.join(df_bed_data.set_index('bed_type'),on = 'bed_type')
df = df.drop('bed_type', axis = 1)


In [ ]:
#continue with cancellation_policy, where we can use: super_strict_60 = 5, super_strict_30 = 4, strict = 3, 
# moderate = 2, flexible = 1, in order of severity
cancel_data = [['super_strict_60',5],['super_strict_30',4],['strict',3],['moderate',2],['flexible',1]]
df_cancel_type = pd.DataFrame(cancel_data,columns=['cancellation_policy','cancellation_val'])
df = df.join(df_cancel_type.set_index('cancellation_policy'),on = 'cancellation_policy')
df = df.drop('cancellation_policy', axis = 1)

In [ ]:
#continue with cleaning_fee, where we can use: True = 1 and False = 0
df['cleaning_fee']= np.where(df['cleaning_fee']== True,1,0)

In [ ]:
#the date of last review and the thumbnail_url will not be used in the price modeling so we drop these columns here
df = df.drop(['thumbnail_url'], axis = 1)
print(len(df.columns))

In [ ]:
#continue with room_type, where we can use: Entire home/apt = 3, Private room = 2, and Shared room = 1
roomtype_data = [['Entire home/apt',3],['Private room',2],['Shared room',1]]
df_roomtype = pd.DataFrame(roomtype_data,columns=['room_type','roomtype_val'])
df = df.join(df_roomtype.set_index('room_type'),on = 'room_type')
df = df.drop('room_type', axis = 1)
df.head()

In [ ]:
#continue with city, where we can create a separate column for each of: Boston, Chicago, DC, LA, NYC, SF

df['Boston'] = (df['city'] == "Boston").astype(int)
df['Chicago'] = (df['city'] == "Chicago").astype(int)
df['DC'] = (df['city'] == "DC").astype(int)
df['LA'] = (df['city'] == "LA").astype(int)
df['NYC'] = (df['city'] == "NYC").astype(int)
df['SF'] = (df['city'] == "SF").astype(int)
df = df.drop('city', axis = 1)
df.head()

In [ ]:
#continue with host_has_profile_pic, Host_identity_verified, and instant_bookable, where we can use: t = 1 and f = 0
df['host_has_profile_pic']= np.where(df['host_has_profile_pic']== 't',1,0)
df['host_identity_verified']= np.where(df['host_identity_verified']== 't',1,0)
df['instant_bookable']= np.where(df['instant_bookable']== 't',1,0)
df.head()

In [ ]:
#continue with host_response_rate, Host_identity_verified, and instant_bookable, where we can use: t = 1 and f = 0
#remove the '%' sign at the end of each entry
df['host_response_rate']= df['host_response_rate'].str.strip('%')
df.head()

In [ ]:
import datetime

#convert dates into 'datetime format'
df['first_review'] = pd.to_datetime(df['first_review'])
df['last_review'] = pd.to_datetime(df['last_review'])

#create a new variable to determine the frequency of listing reviews
#subtract the time between the first and last review and then divide by the number of reviews to get # of days between reviews
df['freq_review'] = df['last_review'].sub(df['first_review'], axis=0)
df['freq_review'] /= np.timedelta64(1, 'D')
df['freq_review'] /= df['number_of_reviews']
df.head()

In [ ]:
#the strategy here is to convert the date the user started hosting to number of days hosting (host experience)
#use January 1, 2018 as a reference date since all data is 2017 or earlier
date_ref = pd.to_datetime('2018-01-01')

#convert 'host_since' column to date_time
df['host_since'] = pd.to_datetime(df['host_since'])

#subtract 'host_since' from reference date
df['host_since'] = date_ref - df['host_since']
df['host_since'] /= np.timedelta64(1, 'D')
df.head()

In [ ]:
df.columns

## Sree's Parts

In [ ]:
# resets index since we dropped rows and I need to use loops referencing the indices
df = df.reset_index().drop(columns = ['index'])

In [ ]:
# cleaning amenities list to remove '{','}'
# standardizing to single quotes
# creating a list of all amenities
all_amenities = [df['amenities'][i].replace("{","").replace("}","").replace('"', '').split(",") for i in range(len(df['amenities']))]

In [ ]:
# identifying unique amenities across all rows of the dataframe
unique_amenities = pd.unique([val for sublist in all_amenities for val in sublist]).tolist()
unique_amenities.remove('')

In [ ]:
# creating combined dataframe to include each of the unique amenities as the columns
df = pd.concat([df,pd.DataFrame(columns = unique_amenities)], sort=False)
df[unique_amenities] = 0

### This function does one hot encoding for amenities for each property (row in the dataframe).

In [ ]:
# function obtains list of amenities for each row
# function looks for individual amenity name in dataframe columns
# function assigns 1 to that amenity column for that row if amenity is present in list of amenities for that rowa
def apply_amenities(df_row):
    clean_list = df_row['amenities'].replace("{","").replace("}","").replace('"', '').split(',')
    for i in range(len(clean_list)):
        df_row[clean_list[i]] = 1
    return df_row

In [ ]:
#apply function onto dataframe
df = df.apply(apply_amenities, axis = 1).drop(columns=[''], axis = 1)

In [ ]:
df